In [3]:
import numpy as np
from sklearn import linear_model
import time

X = np.array([[ 1, 4,90],
              [10, 5,8],
              [ 3, 9,10],
             [23,23,234]])
Y=np.array([1,2,3,10])
X[:,0]

array([ 1, 10,  3, 23])

In [70]:
X[:,[2,1]]

array([[ 90,   4],
       [  8,   5],
       [ 10,   9],
       [234,  23]])

In [19]:
(1,2,3,4)

(1, 2, 3, 4)

In [41]:
a=(i for i in range(10))
for i in a:
    i=i+1
    print(i)
next(a)

1
2
3
4
5
6
7
8
9
10


StopIteration: 

In [99]:
a={1:2,23:3}
a[1+23]=0
a

{1: 2, 23: 3, 24: 0}

In [53]:
a=[i for i in range(10)]
a.index(max(a))

9

In [4]:
# this piece of code is copied directly from the blog.
import time
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print (  '{}  {} ms'.format(method.__name__, (te - ts) * 1000) )
        return result
    return timed

In [5]:
# Simple Least Squared Method Normal Equation
#Linear Regression 
class LinearRegression():
    def __init__(self,Y,X):
        #Y is an vecotor, X is a matrix, they are numpy object
        self.nobs=Y.shape[0]
        self.nvar=X.shape[1]+1 # intercept included
        if self.nobs<self.nvar:
            raise ValueError("degree of freedom is less than 0, impossible to estimate all parameters")
        else: pass
        if np.linalg.matrix_rank(X)==min(X.shape):
            # full rank 
            pass
        else:
            raise ValueError("matrix X does not have full rank.Only non-singular matrix apply this method.")
        
        if Y.shape[0]==X.shape[0]: 
            self.Y=Y
            self.X=np.insert(X,0,1,axis=1)
            self.coe=None
        else: raise ValueError("their dims doesn't match. nrow of Y={}, nrow of X={}".format(Y.shape[0],X.shape[0]))
            
    def __str__(self):
        return("Linear Regression Model: \nnumber of observations={}\nnumber of variables={} (intercept included)".format(self.Y.shape[0],self.X.shape[1]))
    def __repr__(self):
        return(self.__str__())
   
#     @timeit
    def OLS(self,X=None):
        ''' OLS normal equation method ： this formular require X to be full rank '''  
        X= self.X if X is None else X
        XtX_inv=np.linalg.inv(np.dot(X.T,X))
        self.coe=np.dot(np.dot(XtX_inv,X.T),Y)
        return(self.coe)
    
    def GetResidual(self,coe=None):
        coe = self.coe if coe is None else coe
        if coe is None:
            coe = LinearRegression.OLS(self)  
        return(self.Y-np.dot(self.X,coe))
    
mymodel=LinearRegression(Y=Y,X=X)
mymodel.GetResidual()

array([  1.07691633e-14,  -8.88178420e-15,   1.37667655e-14,
        -5.32907052e-15])

In [49]:
mymodel.GetResidual(coe=[1,1,1,1])

array([ -95,  -22,  -20, -271])

In [76]:
a=[1,2,123,3]
2 in a

True

In [68]:
a.remove(123)
a

[1, 2, 3]

In [6]:
class ForwardSelection(LinearRegression):
    '''step-wise selection of variables, each time put the most correlated variable into the model   '''
    def __init__(self,Y,X):
        LinearRegression.__init__(self,Y=Y,X=X)
        self.active=[]
        self.inactive=[i for i in range(self.nvar)]
        self.coeset={0:np.array([0 for i in range(self.nvar)])}# initialize the first round coe
        
    def printnvar(self):
        print(self.active)
        print(self.coeset)
    def stepwise(self):
        for j in range(self.nvar):
            last_residual=self.GetResidual(coe=self.coeset[j]) # residual from last time
            list_of_cov=[np.dot(self.X[:,i],last_residual) for i in self.inactive]  # see the all the covariance of redisual and inactive variables
#             print("this time we have the leftinactive coes{}".format(list_of_cov))
            nextvar=self.inactive[list_of_cov.index(max(list_of_cov))]
#             print("inactive {}  ready to put nextvar {} into active active {} ".format(self.inactive,nextvar,self.active))
            self.active.append(nextvar)
#             print("inactive {} active {} ready to remove nextvar {}".format(self.inactive,self.active,nextvar))
            self.inactive.remove(nextvar)
#             print("now finally inactive {} active {} ".format(self.inactive,self.active))
            # update the coefficients
            X=self.X[:,self.active]# subset the X picking the var in active set
            newcoe=ForwardSelection.OLS(self,X=X)# use this slice of X to get coe
            finalcoe=[None for i in range(self.nvar)]
            for i in range(len(finalcoe)):
                finalcoe[i]= newcoe[self.active.index(i)] if i in self.active else 0
            self.coeset[j+1]=finalcoe
        return(self.coeset)
           
               
                
        
mymodel=ForwardSelection(Y=Y,X=X)
mymodel.stepwise()

{0: array([0, 0, 0, 0]),
 1: [0, 0, 0, 0.039289114566804187],
 2: [0, 0.28143477550791052, 0, 0.014361013695887676],
 3: [0, 0.087098869903475748, 0.31256846241576219, 0.0024502078212224887],
 4: [-0.66389073950699551,
  0.075199866755496447,
  0.38124583610926144,
  0.00070786142571611574]}

In [119]:
mymodel.OLS()

array([-0.66389074,  0.07519987,  0.38124584,  0.00070786])

In [94]:
index=[12,3,4,1]
coe=[1.234,123.34,23.34,4.09]
a=[0 for i in range(len(index))]
for i in range(len(index)):
    a[i]=coe[index.index(i)] if i in index else 0
a

[0, 4.09, 0, 123.34]

In [176]:
reg = linear_model.LinearRegression()
start_time = time.time()
reg.fit (X, Y)
print(reg.coef_)
print("--- %s seconds ---" % (time.time() - start_time))

[ 0.07519987  0.38124584  0.00070786]
--- 0.0026426315307617188 seconds ---


In [9]:
import pandas as pd
import os
# train = pd.read_csv("C:\\Users\\Wesle\\python_project\\train.csv")
ep=os.path.expanduser('~/python_project/train.csv')
os.path.normpath(ep)
train = pd.read_csv(os.path.normpath(ep))

C:\Wesley_Tao\6.Software\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
sample=train.sample(frac=0.01)
sample.head()

,id,target,var1,var2,var3,var4,var5,var6,var7,var8,...,weatherVar227,weatherVar228,weatherVar229,weatherVar230,weatherVar231,weatherVar232,weatherVar233,weatherVar234,weatherVar235,weatherVar236
160933,320786,0.0,Z,Z,Z,H1,Z,Z,4,2,...,0.138410,0.0,0.00000,0.000000,0.689829,1.218189,0.241985,2.891132,0.404655,0.069598
400966,800323,0.0,Z,Z,Z,L1,Z,Z,7,1,...,0.670756,0.0,2.73363,1.000553,0.127746,1.028904,0.645856,1.338487,0.924926,0.062796
23538,46668,0.0,Z,Z,Z,N1,Z,Z,2,3,...,0.255526,0.0,0.00000,0.876535,0.013626,1.124578,0.872398,1.124329,0.289039,0.002641
443489,885649,0.0,2,Z,Z,D4,Z,Z,3,4,...,0.404583,0.0,0.00000,1.658480,0.437744,0.721676,0.499489,0.160618,0.057808,0.129158
193605,386429,0.0,Z,Z,Z,M1,Z,Z,5,2,...,0.383289,0.0,0.00000,0.000000,0.262306,1.165610,0.520025,1.338487,0.520271,0.073295


In [92]:
X=sample.filter(regex="weatherVar[0-9]{1}$")
X.head()

,weatherVar1,weatherVar2,weatherVar3,weatherVar4,weatherVar5,weatherVar6,weatherVar7,weatherVar8,weatherVar9
160933,5.364591,0.0,0.914953,0.728800,0.411371,0.624363,0.722766,7.886224,0.000000
400966,0.670359,0.0,0.969861,0.172978,1.225274,0.096530,1.009153,0.030262,0.000000
23538,7.951660,0.0,0.882133,1.635643,0.420159,0.923142,0.663614,9.497982,28.999883
443489,1.899472,0.0,0.979031,0.861521,1.236062,0.433009,0.989619,0.000000,0.000000
193605,1.742976,0.0,1.004883,0.208295,1.106129,0.070478,1.042800,0.417269,0.000000


In [94]:
X=sample.filter(regex="weatherVar[0-9]{1}$").values
Y=sample['target'].values
sampledata=np.insert(X,0,Y,axis=1)
sampledata=sampledata[~np.isnan(sampledata).any(1)]
# X=X[~np.isnan(X).any(1)]
# X.shape
Y=sampledata[:,0]
X=sampledata[:,1:]
X.shape

(4488, 9)

In [63]:
pd.DataFrame(sampledata).to_csv("sampledata.csv")


In [36]:
import pandas as pd 
df = pd.DataFrame(np_array)
df.to_csv("file_path.csv")

array([ 0.85525321,  0.        ,  0.85729521, ...,  0.26769738,
        0.2312315 ,  0.45958218])

In [95]:
reg = linear_model.LinearRegression()
start_time = time.time()
reg.fit (sampledata[:,1:], sampledata[:,0])
print(reg.coef_)
print("--- %s seconds ---" % (time.time() - start_time))

[  2.38684022e-05   5.68846609e-05   7.40580908e-03  -7.36000130e-05
   2.64308171e-03   2.64453703e-05  -8.26176461e-03  -5.00243457e-05
  -6.33878062e-05]
--- 0.007016897201538086 seconds ---


In [96]:
mymodel=LinearRegression(Y=sampledata[:,0],X=sampledata[:,1:])
mymodel.OLS()

OLS  13.03553581237793 ms


array([ -8.02586730e-04,   2.38684022e-05,   5.68846609e-05,
         7.40580908e-03,  -7.36000130e-05,   2.64308171e-03,
         2.64453703e-05,  -8.26176461e-03,  -5.00243457e-05,
        -6.33878062e-05])

In [97]:
reg.coef_-mymodel.OLS()[1:]

OLS  2.0062923431396484 ms


array([  3.37796739e-18,  -6.77626358e-21,   9.28077060e-17,
         2.94089839e-18,   9.10729825e-18,  -2.43945489e-19,
         3.29597460e-17,   1.00288701e-18,  -2.30392962e-19])